In [22]:
import operator
from math import log
def createData():  
    dataSet=[['20-29','男','10-19','否','否'],['40+','女','30-39','是','是'],['30-39','男','10-19','否','否'],['20-29','男','40+','否','是'],['40+','男','30-39','是','否'],['40+','男','30-39','是','是'],['30-39','男','10-19','否','否'],['20-29','女','30-39','是','否']]
    label=['年龄','性别','收入（万）','婚姻状况']
    return dataSet,label
def calcshan(dataSet):   
    lenDataSet=len(dataSet)
    p={}
    H=0.0
    for data in dataSet:
        currentLabel=data[-1] 
        if currentLabel not in p.keys():  
            p[currentLabel]=0
        p[currentLabel]+=1 
    for key in p:
        px=float(p[key])/float(lenDataSet)  
        H-=px*log(px,2)  
    return H
def spiltData(dataSet,axis,value): 
    subDataSet=[]
    for data in dataSet:
        subData=[]
        if data[axis]==value:
            subData=data[:axis]  
            subData.extend(data[axis+1:]) 
            subDataSet.append(subData) 
    return subDataSet
    
def chooseBestFeature(dataSet):  
    lenFeature=len(dataSet[0])-1
    shanInit=calcshan(dataSet)  
    feature=[]
    inValue=0.0
    bestFeature=0
    for i in range(lenFeature):
        shanCarry=0.0
        feature=[example[i] for example in dataSet]  
        feature=set(feature)  
        for feat in feature:  
            subData=spiltData(dataSet,i,feat)  
            prob=float(len(subData))/float(len(dataSet))
            shanCarry+=prob*calcshan(subData)  
        outValue=shanInit-shanCarry  
        if (outValue>inValue):
            inValue=outValue  
            bestFeature=i
    return bestFeature

def majorCount(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():  
            classCount[vote]=0
        classCount[vote]+=1     
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]  

def createTree(dataSet,label):    
    classList=[example[-1] for example in dataSet]  
    if classList.count(classList[0])==len(classList): 
        return classList[0]
    if len(dataSet[0])==1:   
        return majorCount(classList)
    featBest=chooseBestFeature(dataSet)  
    feature=[example[featBest] for example in dataSet]  
    featValue=set(feature)  
    newLabel=label[featBest]
    del(label[featBest])
    Tree={newLabel:{}}  
    for value in featValue:
        subLabel=label[:]
        Tree[newLabel][value]=createTree(spiltData(dataSet,featBest,value),subLabel)
    return Tree

def classify(tree,label,testVec):   
    firstFeat=list(tree.keys())[0]            
    secondDict=tree[firstFeat]          
    labelIndex=label.index(firstFeat)  
    global classLabel
    for key in secondDict.keys():     
        if testVec[labelIndex]==key:    
            if type(secondDict[key]).__name__=='dict':  
                classLabel=classify(secondDict[key],label,testVec)  
            else:
                classLabel=secondDict[key]  
    return classLabel

def storeTree(tree,filename):  
    import pickle
    fw=open(filename,'wb')
    pickle.dump(tree,fw)
    fw.close()
def loadTree(filename):         
    import pickle
    fr=open(filename,'rb')
    return pickle.load(fr)

if __name__=='__main__':
    dataSet,label=createData()
    labelTree=label[:] 
    tree=createTree(dataSet,labelTree) 
    print(tree)
    print(list(tree.keys()))
    classLabel=classify(tree,label,[39,'男',12,'是','是']) 
    print (classLabel)
    storeTree(tree,'dataTree.txt')
    print (loadTree('dataTree.txt'))

{'收入（万）': {'10-19': '否', '40+': '是', '30-39': {'年龄': {'20-29': '否', '40+': {'性别': {'女': '是', '男': {'婚姻状况': {'是': '否'}}}}}}}}
['收入（万）']
否
{'收入（万）': {'10-19': '否', '40+': '是', '30-39': {'年龄': {'20-29': '否', '40+': {'性别': {'女': '是', '男': {'婚姻状况': {'是': '否'}}}}}}}}
